# 在多 Agent 应用程序中添加多轮对话
我们将创建 2 个智能体：
- travel_advisor：可以提供旅行目的地推荐。可以向 hotel_advisor 寻求帮助。
- hotel_advisor：可以提供酒店推荐。可以向 travel_advisor 寻求帮助。

In [1]:
import operator
import os
import sys
import re
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from utils.env_util import *
from langgraph_utils.common_util import gen_mermaid
import random
from typing import Annotated, Literal

from langchain_core.tools import tool
from langchain_core.tools.base import InjectedToolCallId
from langgraph.prebuilt import InjectedState
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI


@tool
def get_travel_recommendations():
    """Get recommendation for travel destinations"""
    return random.choice(["aruba", "turks and caicos"])


@tool
def get_hotel_recommendations(location: Literal["aruba", "turks and caicos"]):
    """Get hotel recommendations for a given destination."""
    return {
        "aruba": [
            "The Ritz-Carlton, Aruba (Palm Beach)"
            "Bucuti & Tara Beach Resort (Eagle Beach)"
        ],
        "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"],
    }[location]


def make_handoff_tool(*, agent_name: str):
    """ 用于创建一个特殊的工具（tool），该工具允许一个代理（agent）将控制权转移（handoff）给另一个代理 """

    tool_name = f"transfer_to_{agent_name}"

    # 工具的名称动态生成，格式为 transfer_to_{agent_name}，例如 transfer_to_agent2
    @tool(tool_name, description="Create a tool that can return handoff via a Command")
    def handoff_to_agent(
        # InjectedState：管理工具调用过程中的动态上下文，适合跨步骤数据共享。
        state: Annotated[dict, InjectedState],
        # InjectedToolCallId：唯一标识工具调用，用于跟踪和匹配异步操作。
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        # 创建一个工具消息（tool_message），内容为“成功切换到 {agent_name}”，表示控制权已成功转移。
        # 返回一个 Command 对象，用于导航到目标代理节点 """
        tool_message = {
            "role": "tool",
            "content": f"成功切换到 {agent_name}",
            "name": tool_name,
            "tool_call_id": tool_call_id,
        }

        # goto=agent_name: 指定要转移到的目标代理名称。
        # graph=Command.PARENT: 表示在父图中进行导航（即多代理对话的顶层图）。
        # update: 更新目标代理的状态。这里将当前代理的消息历史（state["messages"]）与工具消息（tool_message）合并，确保聊天记录完整有效。
        return Command(
            goto=agent_name,
            graph=Command.PARENT,
            update={"messages": state["messages"] + [tool_message]},
        )

    return handoff_to_agent

In [2]:
model = ChatOpenAI(
    openai_api_key=get_openai_api_key(),
    model_name=get_default_model(),
    # model_name='THUDM/GLM-Z1-32B-0414',
    base_url=get_openai_base_url(),
)

# Define travel advisor tools and ReAct agent
travel_advisor_tools = [
    get_travel_recommendations,
    make_handoff_tool(agent_name="hotel_advisor"),
]
travel_advisor = create_react_agent(
    model,
    travel_advisor_tools,
    prompt=(
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "You MUST include human-readable response before transferring to another agent."
    ),
)


def call_travel_advisor(
    state: MessagesState,
) -> Command[Literal["hotel_advisor", "human"]]:
    # You can also add additional logic like changing the input to the agent / output from the agent, etc.
    # NOTE: we're invoking the ReAct agent with the full history of messages in the state
    response = travel_advisor.invoke(state)
    return Command(update=response, goto="human")

# Define hotel advisor tools and ReAct agent
hotel_advisor = create_react_agent(
    model = model,
    tools = [get_hotel_recommendations, make_handoff_tool(agent_name="travel_advisor")],
    prompt=(
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
        "You MUST include human-readable response before transferring to another agent."
    ),
)


def call_hotel_advisor(
    state: MessagesState,
) -> Command[Literal["travel_advisor", "human"]]:
    response = hotel_advisor.invoke(state)
    return Command(update=response, goto="human")


def human_node(
    state: MessagesState, config
) -> Command[Literal["hotel_advisor", "travel_advisor", "human"]]:

    user_input = interrupt(value="Ready for user input.")

    # identify the last active agent
    # (the last active node before returning to human)
    langgraph_triggers = config["metadata"]["langgraph_triggers"]
    print(langgraph_triggers)
    if len(langgraph_triggers) != 1:
        raise AssertionError("Expected exactly 1 trigger in human node")

    active_agent = langgraph_triggers[0].split(":")[1]

    return Command(
        update={
            "messages": [
                {
                    "role": "human",
                    "content": user_input,
                }
            ]
        },
        goto=active_agent,
    )


builder = StateGraph(MessagesState)
builder.add_node("travel_advisor", call_travel_advisor)
builder.add_node("hotel_advisor", call_hotel_advisor)

# This adds a node to collect human input, which will route
# back to the active agent.
builder.add_node("human", human_node)

# We'll always start with a general travel advisor.
builder.add_edge(START, "travel_advisor")


checkpointer = MemorySaver()
graph = builder.compile(checkpointer=checkpointer)

gen_mermaid(graph, "multi_agent_multi_turn_convo.mmd")

🙈 OPENAI_API_KEY: 5ca39a6a-d****************7dfa221f3f
👀 DEFAULT_MODEL: doubao-1-5-thinking-pro-250415
👀 OPENAI_BASE_URL: https://ark.cn-beijing.volces.com/api/v3
✏️ 已生成 mermaid 文件 /workspace/Agent/langgraph_demo/resources/multi_agent_multi_turn_convo.mmd


In [3]:
import uuid

thread_config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = [
    # 1st round of conversation,
    {
        "messages": [
            {"role": "user", "content": "i wanna go somewhere warm in the caribbean"}
        ]
    },
    # Since we're using `interrupt`, we'll need to resume using the Command primitive.
    # 2nd round of conversation,
    Command(
        resume="could you recommend a nice hotel in one of the areas and tell me which area it is."
    ),
    # 3rd round of conversation,
    Command(
        resume="i like the first one. could you recommend something to do near the hotel?"
    ),
]

for idx, user_input in enumerate(inputs):
    print()
    print(f"--- Conversation Turn {idx + 1} ---")
    print()
    print(f"User: {user_input}")
    print()
    for update in graph.stream(user_input, config=thread_config, stream_mode="updates"):
        for node_id, value in update.items():
            if isinstance(value, dict) and value.get("messages", []):
                last_message = value["messages"][-1]
                if isinstance(last_message, dict) or last_message.type != "ai":
                    continue
                print(f"{node_id}: {last_message.content}")
    # print(graph.get_state(thread_config).next)



--- Conversation Turn 1 ---

User: {'messages': [{'role': 'user', 'content': 'i wanna go somewhere warm in the caribbean'}]}



hotel_advisor: 

在特克斯和凯科斯群岛，为您推荐以下优质酒店：

1. **Grace Bay Club**：坐落在著名的格雷斯湾（Grace Bay）海滩旁，这里以细腻的白沙滩和清澈的 turquoise 海水闻名。酒店提供私人别墅和套房，配备完善的设施，适合家庭或情侣度假，周边还有丰富的水上活动可选。

2. **COMO Parrot Cay**：一个高端私密的度假村，位于帕罗特凯岛（Parrot Cay），主打宁静与自然。酒店设计融合了现代与热带风格，提供水疗、瑜伽课程以及定制化的餐饮体验，是追求放松和奢华的理想之选。

如果需要更详细的酒店信息（如价格、设施或用户评价），可以告诉我，我会尽力为您补充！
('human',)

--- Conversation Turn 2 ---

User: Command(resume='could you recommend a nice hotel in one of the areas and tell me which area it is.')

('branch:to:human',)
()

--- Conversation Turn 3 ---

User: Command(resume='i like the first one. could you recommend something to do near the hotel?')

()
